Good references:
* https://stackoverflow.com/questions/21972690/beautifulsoup-scraping-a-forum-page
* 

todo:
* X scrape: post title, views, number of posts, votes, freshness
* X include original link to post
* parse title using text analysis to get most popular terms over time
* funny posts
* economic related post vs non-econ. related post
* troll vs non-troll

In [23]:
from bs4 import BeautifulSoup
import csv
import urllib2

In [34]:
def ejmr_scrape(pg_num):

    if pg_num % 100 == 0:
        
        print "Webpage number: {0}".format(pg_num)
    
    page = urllib2.urlopen("https://www.econjobrumors.com/page/{0}".format(pg_num))
    soup = BeautifulSoup(page)

    table = soup.find("table", id="latest").find_all("tr") # table containing posts

    ejmr_filename = "ejmr_data/ejmr_pg{0}.csv".format(pg_num)

    header_row = ["title",
                  "num_posts",
                  "views",
                  "votes",
                  "freshness"
                 ]

    with open(ejmr_filename, "wb") as f:

        writeFile = csv.writer(f, quoting=csv.QUOTE_NONNUMERIC)

        writeFile.writerow(header_row)

        for tb in table:

            row = []

            for elem in tb.find_all("td"):

                # remove empty element from list
                elem_list = [e.encode('utf-8').strip() for e in elem]
                try:
                    elem_list = filter(None, elem_list)

                except:

                    print "no blank"

                # get only first element of list (addresses case where title includes page number links next to them---e.g. Top theorists graduating next year? - 23…8910)
                elem_list_first = elem_list[0]
                row.append(elem_list_first)

            if row:  # write row only if non-empty
                writeFile.writerow(row)

In [ ]:
pg_start = 2
pg_end = 1000

pg_start = 1--
pg_end = 2000

for pg_num in xrange(pg_start, pg_end):
    ejmr_scrape(pg_num)

Webpage number: 2
Webpage number: 3
Webpage number: 4
Webpage number: 5
Webpage number: 6
Webpage number: 7
Webpage number: 8
Webpage number: 9
Webpage number: 10
Webpage number: 11
Webpage number: 12
Webpage number: 13
Webpage number: 14
Webpage number: 15
Webpage number: 16
Webpage number: 17
Webpage number: 18
Webpage number: 19
Webpage number: 20
Webpage number: 21
Webpage number: 22
Webpage number: 23
Webpage number: 24
Webpage number: 25
Webpage number: 26
Webpage number: 27
Webpage number: 28
Webpage number: 29
Webpage number: 30
Webpage number: 31
Webpage number: 32
Webpage number: 33
Webpage number: 34
Webpage number: 35
Webpage number: 36
Webpage number: 37
Webpage number: 38
Webpage number: 39
Webpage number: 40
Webpage number: 41
Webpage number: 42
Webpage number: 43
Webpage number: 44
Webpage number: 45
Webpage number: 46
Webpage number: 47
Webpage number: 48
Webpage number: 49
Webpage number: 50
Webpage number: 51
Webpage number: 52
Webpage number: 53
Webpage number: 54
W